# Задача 1: Работа с PostgreSQL

## Импорт необходимых библиотек

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

## Подключение к PostgreSQL

In [7]:
# Параметры подключения
db_params = {
    "dbname": "postgres",
    "user": "admin",
    "password": "postgres",
    "host": "localhost",
    "port": 5432
}

# Создание подключения
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")
conn = engine.connect()

## Создание таблиц и загрузка данных

In [12]:
# Определение SQL-запросов для создания таблиц
create_table_queries = {
    'brands': """
        CREATE TABLE IF NOT EXISTS brands (
            brand_id SERIAL PRIMARY KEY,
            brand_name VARCHAR(255) NOT NULL
        );
    """,
    'categories': """
        CREATE TABLE IF NOT EXISTS categories (
            category_id SERIAL PRIMARY KEY,
            category_name VARCHAR(255) NOT NULL
        );
    """,
    'customers': """
        CREATE TABLE IF NOT EXISTS customers (
            customer_id SERIAL PRIMARY KEY,
            first_name VARCHAR(255) NOT NULL,
            last_name VARCHAR(255) NOT NULL,
            email VARCHAR(255) UNIQUE NOT NULL
        );
    """,
    'order_items': """
        CREATE TABLE IF NOT EXISTS order_items (
            order_item_id SERIAL PRIMARY KEY,
            order_id INT NOT NULL,
            product_id INT NOT NULL,
            quantity INT NOT NULL,
            price NUMERIC(10, 2) NOT NULL
        );
    """,
    'orders': """
        CREATE TABLE IF NOT EXISTS orders (
            order_id SERIAL PRIMARY KEY,
            customer_id INT NOT NULL,
            order_date TIMESTAMP NOT NULL,
            status VARCHAR(50) NOT NULL
        );
    """,
    'products': """
        CREATE TABLE IF NOT EXISTS products (
            product_id SERIAL PRIMARY KEY,
            product_name VARCHAR(255) NOT NULL,
            price NUMERIC(10, 2) NOT NULL,
            category_id INT
        );
    """,
    'staffs': """
        CREATE TABLE IF NOT EXISTS staffs (
            staff_id SERIAL PRIMARY KEY,
            first_name VARCHAR(255) NOT NULL,
            last_name VARCHAR(255) NOT NULL,
            email VARCHAR(255) UNIQUE NOT NULL,
            store_id INT
        );
    """,
    'stocks': """
        CREATE TABLE IF NOT EXISTS stocks (
            stock_id SERIAL PRIMARY KEY,
            store_id INT NOT NULL,
            product_id INT NOT NULL,
            quantity INT NOT NULL
        );
    """,
    'stores': """
        CREATE TABLE IF NOT EXISTS stores (
            store_id SERIAL PRIMARY KEY,
            store_name VARCHAR(255) NOT NULL,
            location VARCHAR(255) NOT NULL
        );
    """
}

# Список файлов и таблиц
csv_files = {
    'brands': 'brands.csv',
    'categories': 'categories.csv',
    'customers': 'customers.csv',
    'order_items': 'order_items.csv',
    'orders': 'orders.csv',
    'products': 'products.csv',
    'staffs': 'staffs.csv',
    'stocks': 'stocks.csv',
    'stores': 'stores.csv'
}

# Создание таблиц и наполнение их данными
for table_name, create_query in create_table_queries.items():
    csv_file_path = csv_files[table_name]
    print(f"Создание таблицы {table_name}...")
    
    # Создание таблицы
    with engine.connect() as conn:
        conn.execute(text(create_query))
        conn.commit()
    print(f"Таблица {table_name} успешно создана или уже существует.")
    
    # Загрузка данных из CSV
    try:
        data = pd.read_csv(csv_file_path)
        if data.empty:
            print(f"Файл {csv_file_path} пуст. Пропуск загрузки данных.")
            continue
        
        # Загрузка данных в таблицу
        data.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Данные успешно загружены в таблицу '{table_name}'.")
    except FileNotFoundError:
        print(f"Ошибка: Файл '{csv_file_path}' не найден. Пропуск таблицы {table_name}.")
    except Exception as e:
        print(f"Ошибка при загрузке данных в таблицу {table_name}: {e}")


Создание таблицы brands...
Таблица brands успешно создана или уже существует.
Данные успешно загружены в таблицу 'brands'.
Создание таблицы categories...
Таблица categories успешно создана или уже существует.
Данные успешно загружены в таблицу 'categories'.
Создание таблицы customers...
Таблица customers успешно создана или уже существует.
Данные успешно загружены в таблицу 'customers'.
Создание таблицы order_items...
Таблица order_items успешно создана или уже существует.
Данные успешно загружены в таблицу 'order_items'.
Создание таблицы orders...
Таблица orders успешно создана или уже существует.
Данные успешно загружены в таблицу 'orders'.
Создание таблицы products...
Таблица products успешно создана или уже существует.
Данные успешно загружены в таблицу 'products'.
Создание таблицы staffs...
Таблица staffs успешно создана или уже существует.
Данные успешно загружены в таблицу 'staffs'.
Создание таблицы stocks...
Таблица stocks успешно создана или уже существует.
Данные успешно загр

## SQL-запросы

In [14]:
conn = engine.connect()  # Открываем новое соединение

# Список запросов
queries = {
    "Запрос 1: Названия продуктов и бренды": """
        SELECT p.product_name, b.brand_name
        FROM products p
        JOIN brands b ON p.brand_id = b.brand_id;
    """,
    "Запрос 2: Активные сотрудники и магазины": """
        SELECT s.staff_id, s.first_name, s.last_name, st.store_name
        FROM staffs s
        JOIN stores st ON s.store_id = st.store_id
        WHERE s.active = 1;
    """,
    "Запрос 3: Клиенты выбранного магазина": """
        SELECT c.first_name, c.last_name, c.email, c.phone
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.store_id = :store_id;
    """,
    "Запрос 4: Количество продуктов в каждой категории": """
        SELECT category_id, COUNT(*) AS product_count
        FROM products
        GROUP BY category_id;
    """,
    "Запрос 5: Общее количество заказов для каждого клиента": """
        SELECT c.customer_id, COUNT(*) AS order_count
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        GROUP BY c.customer_id;
    """,
    "Запрос 6: Полное имя и общее количество заказов клиентов с хотя бы одним заказом": """
        SELECT c.first_name, c.last_name, COUNT(*) AS order_count
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        GROUP BY c.customer_id, c.first_name, c.last_name
        HAVING COUNT(*) > 0;
    """
}

# Выполнение запросов
with engine.connect() as conn:
    for description, query in queries.items():
        if ":store_id" in query:  # Если запрос содержит параметр :store_id
            store_id = 1  # Укажите здесь значение store_id (замените на ваше значение)
            result = pd.read_sql(text(query), conn, params={"store_id": store_id})
        else:
            result = pd.read_sql(query, conn)
        
        print(f"\n{description}:")
        print(result)



Запрос 1: Названия продуктов и бренды:
                           product_name brand_name
0          Electra Super Moto 8i - 2018    Electra
1     Electra Queen of Hearts 3i - 2018    Electra
2         Electra Tiger Shark 3i - 2018    Electra
3    Electra Cruiser 1 - 2016/2017/2018    Electra
4    Electra Townie Original 21D - 2018    Electra
..                                  ...        ...
316           Trek Lift+ Lowstep - 2018       Trek
317             Trek Dual Sport+ - 2018       Trek
318                   Trek Lift+ - 2018       Trek
319                  Trek XM700+ - 2018       Trek
320                  Trek Verve+ - 2018       Trek

[321 rows x 2 columns]

Запрос 2: Активные сотрудники и магазины:
   staff_id  first_name last_name        store_name
0         4      Virgie   Wiggins  Santa Cruz Bikes
1         3       Genna   Serrano  Santa Cruz Bikes
2         2      Mireya  Copeland  Santa Cruz Bikes
3         1     Fabiola   Jackson  Santa Cruz Bikes
4         7      Veni

### Создание дампа
```sh
PGPASSWORD='postgres' pg_dump -U admin -d postgres -h localhost -p 5432 \
-t customers \
-t staffs \
-t orders \
-t stores \
-t order_items \
-t categories \
-t products \
-t stocks \
-t brands > ~/Desktop/inno_ml/INNO_HW11/dump.sql
```